In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# 0 Load GTFS-based Oslo data (bus part of the multimodal network)

nodes_oslo = pd.read_csv("../../Extraction/out/nodes_GTFS_OSLO.csv")
edges_oslo = pd.read_csv("../../Extraction/out/edges_GTFS_OSLO_with_mondayTrips.csv")

print("Raw input preview:")
display(nodes_oslo.head())
display(edges_oslo.head())

print("\nNodes shape:", nodes_oslo.shape)
print("Edges shape:", edges_oslo.shape)
print("Counts per mode:")
print(edges_oslo["mode"].value_counts())

Raw input preview:


,id,stopPlaceId,name,lat,lon,modes,stopType
0,NSR:Quay:99688,NSR:Quay:99688,Vækerø,59.915650,10.651833,"bus,coach service",multimodal
1,NSR:Quay:11880,NSR:Quay:11880,Maritim,59.918496,10.667065,"bus,coach service",multimodal
2,NSR:Quay:11829,NSR:Quay:11829,Sjølyst,59.920456,10.676683,"bus,coach service",multimodal
3,NSR:Quay:7700,NSR:Quay:7700,Vika atrium,59.910762,10.722139,"bus,coach service",multimodal
4,NSR:Quay:11969,NSR:Quay:11969,Oslo bussterminal,59.911940,10.756710,bus,bus


,from,to,lineId,lineCode,mode,authority,travelTimeSec,tripsInFeed,tripsOn2025_11_17
0,NSR:Quay:99688,NSR:Quay:11880,BRA:Line:4_6169,169,bus,Brakar,60.0,7,14
1,NSR:Quay:11880,NSR:Quay:11829,BRA:Line:4_6169,169,bus,Brakar,60.0,7,14
2,NSR:Quay:11829,NSR:Quay:7700,BRA:Line:4_6169,169,bus,Brakar,120.0,7,14
3,NSR:Quay:7700,NSR:Quay:11969,BRA:Line:4_6169,169,bus,Brakar,660.0,7,14
4,NSR:Quay:12002,NSR:Quay:7699,BRA:Line:4_6169,169,bus,Brakar,660.0,7,14



Nodes shape: (1941, 7)
Edges shape: (4825, 9)
Counts per mode:
mode
bus              3896
metro             372
tram              300
rail              194
coach service      49
unknown            14
Name: count, dtype: int64


In [3]:
# 1 Filter bus edges only

bus_edges = edges_oslo[edges_oslo["mode"] == "bus"].copy()

print("\nBus edges shape:", bus_edges.shape)
print("\nBus edges dtypes:")
print(bus_edges.dtypes)

print("\nMissing values in raw bus_edges:")
print(bus_edges.isna().sum())


Bus edges shape: (3896, 9)

Bus edges dtypes:
from                  object
to                    object
lineId                object
lineCode              object
mode                  object
authority             object
travelTimeSec        float64
tripsInFeed            int64
tripsOn2025_11_17      int64
dtype: object

Missing values in raw bus_edges:
from                   0
to                     0
lineId                 0
lineCode               0
mode                   0
authority              0
travelTimeSec        144
tripsInFeed            0
tripsOn2025_11_17      0
dtype: int64


In [4]:
# 2 Clean travelTimeSec on bus edges
# a) convert to numeric
# b) impute NaN within each lineId by median
# c) fallback: global median for any remaining NaN


# 2a Convert to numeric (invalid values -> NaN)
bus_edges["travelTimeSec"] = pd.to_numeric(
    bus_edges["travelTimeSec"],
    errors="coerce",
)

print(
    "\nNumber of missing values in travelTimeSec after to_numeric:",
    bus_edges["travelTimeSec"].isna().sum(),
)


Number of missing values in travelTimeSec after to_numeric: 144


In [5]:
# 2b Per-line median imputation (within each lineId)
bus_edges["travelTimeSec"] = (
    bus_edges
    .groupby("lineId")["travelTimeSec"]
    .transform(lambda x: x.fillna(x.median()))
)

print(
    "Number of missing values after per-line median fill:",
    bus_edges["travelTimeSec"].isna().sum(),
)

Number of missing values after per-line median fill: 2


c:\pr\statistical_lerning\ACIT4620_Exam_Project_Code\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [6]:
# 2c Global median fallback for lines that had all NaN
global_median = bus_edges["travelTimeSec"].median()
bus_edges["travelTimeSec"] = bus_edges["travelTimeSec"].fillna(global_median)

print(
    "Number of missing values after global median fill:",
    bus_edges["travelTimeSec"].isna().sum(),
)

print("\ntravelTimeSec statistics (seconds):")
print(bus_edges["travelTimeSec"].describe())

Number of missing values after global median fill: 0

travelTimeSec statistics (seconds):
count    3896.000000
mean       94.751027
std        66.951515
min        60.000000
25%        60.000000
50%        60.000000
75%       120.000000
max       840.000000
Name: travelTimeSec, dtype: float64


In [7]:
# 3 Prepare bus nodes
# a) keep only nodes that have "bus" in the 'modes' string
# b) keep only nodes that are actually used by bus_edges
# c) ensure lat/lon are numeric

print("\nMissing values in nodes_oslo:")
print(nodes_oslo.isna().sum())

print("\nNodes dtypes:")
print(nodes_oslo.dtypes)


Missing values in nodes_oslo:
id             0
stopPlaceId    0
name           0
lat            0
lon            0
modes          0
stopType       0
dtype: int64

Nodes dtypes:
id              object
stopPlaceId     object
name            object
lat            float64
lon            float64
modes           object
stopType        object
dtype: object


In [8]:
# 3a Filter nodes by mode 'bus'
nodes_oslo["modes"] = nodes_oslo["modes"].astype(str)
bus_nodes = nodes_oslo[nodes_oslo["modes"].str.contains("bus")].copy()
print("\nBus nodes (by modes):", bus_nodes.shape)


Bus nodes (by modes): (1525, 7)


In [9]:
# 3b Keep only nodes that appear as endpoints in bus_edges
used_node_ids = set(bus_edges["from"]) | set(bus_edges["to"])
bus_nodes = bus_nodes[bus_nodes["id"].isin(used_node_ids)].copy()
print("Bus nodes after intersect with bus_edges:", bus_nodes.shape)

Bus nodes after intersect with bus_edges: (1525, 7)


In [10]:
# 3c Ensure lat/lon are numeric
bus_nodes["lat"] = pd.to_numeric(bus_nodes["lat"], errors="coerce")
bus_nodes["lon"] = pd.to_numeric(bus_nodes["lon"], errors="coerce")

print("\nNaN in lat:", bus_nodes["lat"].isna().sum())
print("NaN in lon:", bus_nodes["lon"].isna().sum())


NaN in lat: 0
NaN in lon: 0


In [11]:
# 4 Build coordinate dictionary and Haversine distance function

# Map: node_id -> {"lat": ..., "lon": ...}
node_coords = bus_nodes.set_index("id")[["lat", "lon"]].to_dict("index")


def haversine_m(lat1, lon1, lat2, lon2):
    """
    Compute geographic distance (meters) between two coordinates
    using the Haversine formula (spherical Earth approximation).
    """
    R = 6371000  # Earth's radius in meters

    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = phi2 - phi1
    dlambda = math.radians(lon2 - lon1)

    a = (
        math.sin(dphi / 2) ** 2
        + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c


def edge_distance(row):
    """
    Compute length of a bus edge based on coordinates of 'from' and 'to' stops.
    If any coordinate is missing, return NaN.
    """
    u = row["from"]
    v = row["to"]

    cu = node_coords.get(u)
    cv = node_coords.get(v)

    if cu is None or cv is None:
        return np.nan

    return haversine_m(cu["lat"], cu["lon"], cv["lat"], cv["lon"])

In [12]:
# 5. Compute distance for every bus edge + edge-level speeds

bus_edges["distance_m"] = bus_edges.apply(edge_distance, axis=1)

print("\nMissing values in distance_m:", bus_edges["distance_m"].isna().sum())
print("Distance column statistics (meters):")
print(bus_edges["distance_m"].describe())


Missing values in distance_m: 0
Distance column statistics (meters):
count    3896.000000
mean      571.511519
std       533.342972
min         8.359992
25%       315.178757
50%       434.295317
75%       628.884502
max      7602.944150
Name: distance_m, dtype: float64


In [13]:
# Edge-level speed (mainly for diagnostics / plots)
bus_edges["speed_m_s"] = bus_edges["distance_m"] / bus_edges["travelTimeSec"]
bus_edges["speed_km_h"] = bus_edges["speed_m_s"] * 3.6

print("\nEdge-level speed statistics (km/h):")
print(bus_edges["speed_km_h"].describe())


Edge-level speed statistics (km/h):
count    3896.000000
mean       23.260701
std        13.660228
min         0.501600
25%        14.164325
50%        19.899037
75%        28.149256
max       108.286508
Name: speed_km_h, dtype: float64


In [14]:
# 6. Aggregate to route-level statistics (per lineId)

# Rename tripsOn2025_11_17 -> tripsPerDay for clarity
bus_edges["tripsPerDay"] = bus_edges["tripsOn2025_11_17"].astype(float)

route_stats = (
    bus_edges
    .groupby("lineId")
    .agg(
        lineCode=("lineCode", "first"),
        authority=("authority", "first"),
        num_edges=("from", "count"),
        total_travel_time_sec=("travelTimeSec", "sum"),
        avg_edge_time_sec=("travelTimeSec", "mean"),
        total_distance_m=("distance_m", "sum"),
        avg_edge_distance_m=("distance_m", "mean"),
        trips_per_day=("tripsPerDay", "first"),
    )
    .reset_index()
)

# Commercial (route-level) speed
route_stats["commercial_speed_m_s"] = (
    route_stats["total_distance_m"] / route_stats["total_travel_time_sec"]
)
route_stats["commercial_speed_km_h"] = route_stats["commercial_speed_m_s"] * 3.6

print("\nNumber of bus routes:", route_stats.shape[0])
print("\nFirst few routes with basic stats:")
display(route_stats.head())

print("\nSummary of key route-level metrics:")
print(
    route_stats[
        [
            "total_distance_m",
            "total_travel_time_sec",
            "commercial_speed_km_h",
            "trips_per_day",
        ]
    ].describe()
)


Number of bus routes: 130

First few routes with basic stats:


,lineId,lineCode,authority,num_edges,total_travel_time_sec,avg_edge_time_sec,total_distance_m,avg_edge_distance_m,trips_per_day,commercial_speed_m_s,commercial_speed_km_h
0,BRA:Line:4_6169,169,Brakar,8,1920.0,240.000000,12718.261163,1589.782645,14.0,6.624094,23.846740
1,BRA:Line:4_6200,200,Brakar,10,2940.0,294.000000,18857.621357,1885.762136,57.0,6.414157,23.090965
2,INN:Line:126,126,Innlandstrafikk,67,7740.0,115.522388,49349.418099,736.558479,14.0,6.375894,22.953218
3,NBU:Line:a7c15243-8873-4aee-a8b4-8d22e8ad31dc,FB5,Flybussen Connect,49,5820.0,118.775510,28270.615316,576.951333,47.0,4.857494,17.486979
4,NBU:Line:a7cafc59-feaf-4e61-9fa1-b7dce4ffe970,FB1,Flybussen Connect,56,4740.0,84.642857,30366.260063,542.254644,43.0,6.406384,23.062982



Summary of key route-level metrics:
       total_distance_m  total_travel_time_sec  commercial_speed_km_h  \
count        130.000000             130.000000             130.000000   
mean       17127.760601            2839.615385              24.247448   
std        10272.051748            1911.955208               9.154053   
min          587.446562              60.000000              10.910720   
25%        10006.696181            1260.000000              17.548042   
50%        16558.412669            2490.000000              21.860213   
75%        23390.648707            4065.000000              29.765542   
max        49349.418099            8700.000000              58.055681   

       trips_per_day  
count     130.000000  
mean       80.307692  
std        86.178636  
min         0.000000  
25%         9.250000  
50%        45.000000  
75%       143.250000  
max       363.000000  


In [15]:
# 8 Passenger estimation and load factor 

# 8a Global assumption: total daily bus ridership in Oslo
# From UITP / CityTransit screenshot: ~382,000 average daily bus journeys.
total_estimated_bus_passengers_per_day = 390_000  # adjust if you find a better year

# Make sure trips_per_day is numeric and non-null
route_stats["trips_per_day"] = (
    route_stats["trips_per_day"]
    .astype(float)
    .fillna(0.0)
)

# 8b Service supply per route: vehicle-km per day
# Vehicle_km_per_day = route length (km) * trips_per_day
# Standard metric in transport planning.
route_stats["vehicle_km_per_day"] = (
    route_stats["total_distance_m"] / 1000.0
) * route_stats["trips_per_day"]

total_vehicle_km = route_stats["vehicle_km_per_day"].sum()
print(f"\nTotal vehicle-km per day across all bus routes: {total_vehicle_km:,.1f}")


Total vehicle-km per day across all bus routes: 203,039.7


In [16]:
# 8c Distribute total passengers across routes proportional to vehicle-km.
# Long + frequent routes get more passengers.
route_stats["passengers_est_per_day"] = 0.0
if total_vehicle_km > 0:
    route_stats["passengers_est_per_day"] = (
        route_stats["vehicle_km_per_day"] / total_vehicle_km
        * total_estimated_bus_passengers_per_day
    )

print("\nEstimated passengers per day (vehicle-km method, first 5 routes):")
display(
    route_stats[
        ["lineId", "lineCode", "vehicle_km_per_day", "passengers_est_per_day"]
    ].head()
)

print("\nPassengers_est_per_day summary:")
print(route_stats["passengers_est_per_day"].describe())


Estimated passengers per day (vehicle-km method, first 5 routes):


,lineId,lineCode,vehicle_km_per_day,passengers_est_per_day
0,BRA:Line:4_6169,169,178.055656,342.010447
1,BRA:Line:4_6200,200,1074.884417,2064.644885
2,INN:Line:126,126,690.891853,1327.069505
3,NBU:Line:a7c15243-8873-4aee-a8b4-8d22e8ad31dc,FB5,1328.718920,2552.211826
4,NBU:Line:a7cafc59-feaf-4e61-9fa1-b7dce4ffe970,FB1,1305.749183,2508.091408



Passengers_est_per_day summary:
count      130.000000
mean      3000.000000
std       4271.258483
min          0.000000
25%        116.620924
50%       1017.500235
75%       3741.879041
max      21236.795488
Name: passengers_est_per_day, dtype: float64


In [17]:
# 8d Capacity estimation: effective seats per bus * trips_per_day
# This is conceptually the same as your colleague's capacity_from_mode
# but restricted to buses (one mode multiplier: seats_per_bus).

seats_per_bus = 45   # effective average capacity per departure (approx)
route_stats["capacity_per_day"] = route_stats["trips_per_day"] * seats_per_bus

print("\nCapacity_per_day summary:")
print(route_stats["capacity_per_day"].describe())


Capacity_per_day summary:
count      130.000000
mean      3613.846154
std       3878.038620
min          0.000000
25%        416.250000
50%       2025.000000
75%       6446.250000
max      16335.000000
Name: capacity_per_day, dtype: float64


In [ ]:
# 8e Load factor = estimated passengers / capacity
route_stats["load_factor"] = np.where(
    route_stats["capacity_per_day"] > 0,
    route_stats["passengers_est_per_day"] / route_stats["capacity_per_day"],
    np.nan,
)

print("\nLoad factor summary (vehicle-km based):")
print(route_stats["load_factor"].describe())


Load factor summary (vehicle-km based):
count    108.000000
mean       0.752236
std        0.454439
min        0.025075
25%        0.436071
50%        0.708911
75%        1.015215
max        2.106460
Name: load_factor, dtype: float64


In [19]:
# 8f Sanity-check
print("\nCheck sums:")
print("Sum passengers_est_per_day:", route_stats["passengers_est_per_day"].sum())
print("Sum capacity_per_day:", route_stats["capacity_per_day"].sum())
print(
    "Network-average load factor:",
    route_stats["passengers_est_per_day"].sum()
    / route_stats["capacity_per_day"].sum()
)


Check sums:
Sum passengers_est_per_day: 389999.99999999994
Sum capacity_per_day: 469800.0
Network-average load factor: 0.830140485312899


In [ ]:
# 9 Combined performance score: speed + load factor

# 9a Speed-based score (0–1)
speed_min = route_stats["commercial_speed_km_h"].min()
speed_max = route_stats["commercial_speed_km_h"].max()
route_stats["speed_score"] = (
    (route_stats["commercial_speed_km_h"] - speed_min)
    / (speed_max - speed_min)
)

# 9b Capacity-based score 
ideal_lf = 0.85
sigma = 0.30

route_stats["capacity_score"] = np.exp(
    -((route_stats["load_factor"] - ideal_lf) ** 2) / (2 * sigma**2)
)

# 9c Final route score
w_speed = 0.5
w_capacity = 0.5

route_stats["route_score"] = (
    w_speed * route_stats["speed_score"]
    + w_capacity * route_stats["capacity_score"]
)

print("\nRoute_score summary:")
print(route_stats["route_score"].describe())


Route_score summary:
count    108.000000
mean       0.405454
std        0.203441
min        0.012921
25%        0.235111
50%        0.430421
75%        0.560767
max        0.792714
Name: route_score, dtype: float64


In [21]:
print("\nTop 10 routes by overall route_score:")
display(
    route_stats
    .sort_values("route_score", ascending=False)
    [
        [
            "lineId",
            "lineCode",
            "trips_per_day",
            "vehicle_km_per_day",
            "passengers_est_per_day",
            "capacity_per_day",
            "load_factor",
            "route_score",
            "commercial_speed_km_h",
        ]
    ]
    .head(10)
)


Top 10 routes by overall route_score:


,lineId,lineCode,trips_per_day,vehicle_km_per_day,passengers_est_per_day,capacity_per_day,load_factor,route_score,commercial_speed_km_h
56,RUT:Line:3125,125E,44.0,731.738081,1405.527201,1980.0,0.709862,0.792714,43.383681
32,RUT:Line:2400,400E,22.0,255.444995,490.660386,990.0,0.495617,0.748862,58.055681
81,RUT:Line:400,400,165.0,2958.026446,5681.796175,7425.0,0.765225,0.721211,33.613937
8,RUT:Line:110,110,213.0,4465.475477,8577.313947,9585.0,0.894868,0.696357,29.949534
89,RUT:Line:505,505,90.0,1881.157422,3613.339245,4050.0,0.892183,0.696046,29.859642
62,RUT:Line:3500,500N,6.0,99.515020,191.149089,270.0,0.707960,0.695205,34.315524
88,RUT:Line:500,500,93.0,1542.482811,2962.810880,4185.0,0.707960,0.683074,33.171673
104,RUT:Line:67,67,73.0,1361.488493,2615.155833,3285.0,0.796090,0.680585,28.693119
43,RUT:Line:300,300,229.0,4184.702352,8038.003127,10305.0,0.780010,0.669022,28.113553
55,RUT:Line:3115,115E,36.0,418.647455,804.140718,1620.0,0.496383,0.662471,49.838983


In [22]:
# Save enriched outputs for further use
bus_edges.to_csv("./results/bus_edges_prepared.csv", index=False)
route_stats.to_csv("./results/bus_route_stats.csv", index=False)